# Langchain Q&A with RAG

This notebook demonstrates how to use the Langchain Q&A Chain with RAG to answer questions based on a document. The document is loaded into a HANA table using the Langchain HanaVectorStore. The Q&A Chain uses the RAG model to answer questions based on the document.

You can learn more about the Langchain Q&A Chain with RAG in the [Langchain documentation](https://python.langchain.com/v0.1/docs/use_cases/question_answering/chat_history/).

# IMPORTANT: Before you start

Set your unique id in the cell below. This will be used to create a unique table name in HANA.

Make sure not to have any (leading or trailing) spaces in the unique id. If you have spaces, replace them with underscores.

In [11]:
YOUR_UNIQUE_ID = "<INSERT_YOUR_ID_HERE>".upper()

In [12]:
# Get HANA credentials
import json
try:
    # load keyfile from ../secrets/hana.json
    with open("../secrets/hana.json") as f:
        dbcreds = json.load(f)
        print("Found HANA credentials.")
except:
    print("Failed to load keyfile. Please make sure you have stored the file in a folder called 'secrets' located in project root.")

Found HANA credentials.


In [13]:
# Set up AI Core OpenAI Langchain Proxy connection using generative-ai-hub-sdk
from gen_ai_hub.proxy.langchain.openai import OpenAIEmbeddings
embed = OpenAIEmbeddings(proxy_model_name='text-embedding-3-small')

In [14]:
# Create a connection using hana-ml
from hana_ml import ConnectionContext

# cc = ConnectionContext(userkey='VDB_BETA', encrypt=True) # when using key from hdbuserstore
cc= ConnectionContext(
    address=dbcreds["host"], 
    port=dbcreds["port"], 
    user=dbcreds["user"], 
    password=dbcreds["password"], 
    encrypt=True
    )
connection = cc.connection
hana_schema = cc.get_current_schema()
print(cc.hana_version())
print(hana_schema)

4.00.000.00.1726574991 (fa/CE2024.28)
USR_59GP137CB6UCA1OVEH3DIIEMU


In [15]:
from langchain_community.vectorstores.hanavector import HanaDB

embeddings_table = f"{YOUR_UNIQUE_ID}_LANGCHAIN_RAG_CHAIN"

# creates a table if it does not exists yet
db = HanaDB(
    embedding=embed, connection=connection, table_name=embeddings_table
)

In [16]:
# Delete already existing documents from the table
db.delete(filter={})

True

# Load a PDF document and add it to the table

Important: If you want to use a local document instead of the one downloaded from the web that we are using by default, make sure to have the document available in the data folder. If you want to use a different document, change the file path or web url accordingly.

To add the document to the data folder, create a subfolder called data in the folder where this notebook is located. Then, add the document to the data folder.

Make sure to rename the file or adjust the file path accordingly.

In [17]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# local_file_path = "./data/genaihub_ai_core.pdf"
web_file_path = "https://help.sap.com/doc/c31b38b32a5d4e07a4488cb0f8bb55d9/CLOUD/en-US/f17fa8568d0448c685f2a0301061a6ee.pdf"

loader = PyPDFLoader(web_file_path, extract_images=False)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
text_chunks = loader.load_and_split(text_splitter)

print(text_chunks[0])

print(f"Number of document chunks: {len(text_chunks)}")

page_content='Service Guide | PUBLIC
2024-09-17
SAP AI Core© 2024 SAP SE or an SAP affiliate  company. All rights reserved.
THE BEST RUN' metadata={'source': 'https://help.sap.com/doc/c31b38b32a5d4e07a4488cb0f8bb55d9/CLOUD/en-US/f17fa8568d0448c685f2a0301061a6ee.pdf', 'page': 0}
Number of document chunks: 619


In [18]:
split_chunks = [text_chunks[i:i+100] for i in range(0, len(text_chunks), 100)]
# add the loaded document chunks
for chunks in split_chunks:
    db.add_documents(chunks)

In [19]:
# take a look at the table
hdf = cc.sql(f''' SELECT "VEC_TEXT", "VEC_META", TO_NVARCHAR("VEC_VECTOR") AS "VEC_VECTOR" FROM {embeddings_table} ''')
localdf = hdf.head(10).collect()
localdf

,VEC_TEXT,VEC_META,VEC_VECTOR
0,Service Guide | PUBLIC\n2024-09-17\nSAP AI Cor...,"{""source"": ""https://help.sap.com/doc/c31b38b32...","[-0.01243771,0.026832066,0.014955354,-0.018389..."
1,Content\n1 What Is SAP AI Core? .................,"{""source"": ""https://help.sap.com/doc/c31b38b32...","[-0.009583199,-0.007215989,0.065046825,-0.0143..."
2,4.1 Free Tier ...................................,"{""source"": ""https://help.sap.com/doc/c31b38b32...","[-0.0016880932,-0.009460846,0.06440146,-0.0158..."
3,Enable Cloud Foundry ............................,"{""source"": ""https://help.sap.com/doc/c31b38b32...","[0.011808621,0.016032327,0.063063875,-0.030212..."
4,Create an Application to Sync Y our Folders .....,"{""source"": ""https://help.sap.com/doc/c31b38b32...","[-0.00795204,0.042798124,0.046562128,-0.017425..."
5,7 .4 Manage Object Store Secrets ................,"{""source"": ""https://help.sap.com/doc/c31b38b32...","[0.005675242,0.031929195,0.04001284,-0.0357683..."
6,Update a Generic Secret .........................,"{""source"": ""https://help.sap.com/doc/c31b38b32...","[0.00043142107,-0.009738354,0.08119084,-0.0111..."
7,8.4 Orchestration ...............................,"{""source"": ""https://help.sap.com/doc/c31b38b32...","[0.025592165,-0.01682132,0.05037549,-0.0211309..."
8,Connect Y our Data ..............................,"{""source"": ""https://help.sap.com/doc/c31b38b32...","[0.015408695,0.010766252,0.040982418,-0.028313..."
9,"11.1 Security Features of Data, Data Flow, and...","{""source"": ""https://help.sap.com/doc/c31b38b32...","[0.0040854346,0.017747745,0.099012434,-0.01032..."


In [20]:
countdf = cc.sql(f''' SELECT COUNT(*) FROM {embeddings_table} ''').collect()
countdf

,COUNT(*)
0,619


## Raw similarity search

In [21]:
query = "What is the Orchestration Service"
docs = db.similarity_search_with_relevance_scores(query, k=2, score_threshold=0.1, filter={})

for doc in docs:
    print("-" * 80)
    print("Source: ", doc[0].metadata)
    print(doc[0].page_content)

--------------------------------------------------------------------------------
Source:  {'source': 'https://help.sap.com/doc/c31b38b32a5d4e07a4488cb0f8bb55d9/CLOUD/en-US/f17fa8568d0448c685f2a0301061a6ee.pdf', 'page': 138}
} }
8.4.3  Using the Orchestration Service
8.4.3.1 Get an Auth Token for Orchestration
Using Postman
Prerequisites
•Y ou have downloaded and installed the Postman client from https:/ /www.postman.com/
 .
•Y ou have familiarized yourself with the Postman documentation and interface.
Procedure
1. Download the JSON collection from Orchestration JSON .
2. In Postman, click Import , select the JSON file, and choose Import  to start the import.
3. After the import is complete, expand the collection and navigate to Get Auth T oken .
4. Select the LLM Orchestration  environment within Postman and configure  it using the following values:
•The LLM Orch Url  is the URL of your orchestration deployment. For more information, see Create a
Deployment for Orchestration [page 141]

## Langchain Q&A Chain with RAG

In [31]:
# Create a retriever instance of the vector store
retriever = db.as_retriever()

In [32]:
from langchain_core.prompts import PromptTemplate

prompt_template = """You are an expert in SAP BTP services. You are provided multiple context items (help documentation) that are related to the prompt you have to answer.
Only use the following pieces of context to answer the question at the end. If you don't find an appropriate answer in the given sources, apologize and say you don't know.
Always answer in English.

Context:
'''
{context}
'''

Question: {question}
"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}

In [33]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from gen_ai_hub.proxy.langchain.openai import ChatOpenAI

llm = ChatOpenAI(proxy_model_name="gpt-4o")
memory = ConversationBufferMemory(
    memory_key="chat_history", output_key="answer", return_messages=True
)
qa_chain = ConversationalRetrievalChain.from_llm(
    llm,
    db.as_retriever(
        search_type="similarity_score_threshold", 
        search_kwargs={"k": 5, "filter": {}, "score_threshold": 0.05}
    ),
    return_source_documents=True,
    memory=memory,
    verbose=True,
    combine_docs_chain_kwargs={"prompt": PROMPT},
)

In [34]:
print(memory)

output_key='answer' return_messages=True memory_key='chat_history'


In [35]:
question = "What is the Orchestration Service?"

result = qa_chain.invoke({"question": question})
print("Answer from LLM:")
print("================")
print(result["answer"])

source_docs = result["source_documents"]
print("================")
print(f"Number of used source document chunks: {len(source_docs)}")
for doc in source_docs: print(f"Source: {doc.metadata}") # access content through doc.page_content



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
You are an expert in SAP BTP services. You are provided multiple context items (help documentation) that are related to the prompt you have to answer.
Only use the following pieces of context to answer the question at the end. If you don't find an appropriate answer in the given sources, apologize and say you don't know.
Always answer in English.

Context:
'''
} }
8.4.3  Using the Orchestration Service
8.4.3.1 Get an Auth Token for Orchestration
Using Postman
Prerequisites
•Y ou have downloaded and installed the Postman client from https:/ /www.postman.com/
 .
•Y ou have familiarized yourself with the Postman documentation and interface.
Procedure
1. Download the JSON collection from Orchestration JSON .
2. In Postman, click Import , select the JSON file, and choose Import  to start the import.
3. After the import is complete, expand the collection and navigate to Get Auth T oken .

In [36]:
question = "Which model providers are available in the genai hub?"

result = qa_chain.invoke({"question": question})
print(result["answer"])



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What is the Orchestration Service?
Assistant: The Orchestration Service in SAP BTP allows you to combine different modules into a pipeline that can be executed with a single API call. Each module's response can be used as the input for the next module within the pipeline. This service enables the coordination and configuration of various AI modules to work together seamlessly, ensuring streamlined execution of complex workflows. The orchestration configuration is passed in JSON format with the request body, which allows for customization and omission of optional modules.

For detailed steps on how to use the Orchestration Service, including obtaining an auth token and creating a deployment, you would typically use tools like Postman and follow the outlined proc

In [37]:
question = "Elaborate on the second one you mentioned"

result = qa_chain.invoke({"question": question})
print(result["answer"])



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What is the Orchestration Service?
Assistant: The Orchestration Service in SAP BTP allows you to combine different modules into a pipeline that can be executed with a single API call. Each module's response can be used as the input for the next module within the pipeline. This service enables the coordination and configuration of various AI modules to work together seamlessly, ensuring streamlined execution of complex workflows. The orchestration configuration is passed in JSON format with the request body, which allows for customization and omission of optional modules.

For detailed steps on how to use the Orchestration Service, including obtaining an auth token and creating a deployment, you would typically use tools like Postman and follow the outlined proc

In [38]:
question = "Which scenario and executable do I have to use when I want to use an LLM from the provider we just talked about?"

result = qa_chain.invoke({"question": question})
print(result["answer"])



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What is the Orchestration Service?
Assistant: The Orchestration Service in SAP BTP allows you to combine different modules into a pipeline that can be executed with a single API call. Each module's response can be used as the input for the next module within the pipeline. This service enables the coordination and configuration of various AI modules to work together seamlessly, ensuring streamlined execution of complex workflows. The orchestration configuration is passed in JSON format with the request body, which allows for customization and omission of optional modules.

For detailed steps on how to use the Orchestration Service, including obtaining an auth token and creating a deployment, you would typically use tools like Postman and follow the outlined proc